In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

tf.keras.backend.clear_session()

In [2]:
from src.models import imagenet_ontop_model
from src import custom_losses, custom_metrics, optimizers
from src.data import data

Using TensorFlow backend.


In [3]:
batch_size = 128
n_classes = 6
epochs = 64
img_size = 224
n_channels = 3

In [4]:
model = imagenet_ontop_model.ImageNetOntop_Model(batch_size, n_classes, epochs, img_size, n_channels, version=4)

W0111 10:55:53.191709 140571796690752 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [5]:
from src.data import data

In [6]:
paths = data.PATH()
dataset_path = f'{paths.PROCESSED_DATA_PATH}/'
dataset = 'vision_based_dataset'
test_dataset_path = f'{dataset_path}/{dataset}/'

In [7]:
train_generator, validation_generator, test_generator = model.get_image_data_generator(test_dataset_path, train=True, validation=True, test=True, class_mode_validation='categorical', class_mode_test='categorical')

Found 114361 images belonging to 6 classes.
Found 6305 images belonging to 6 classes.
Found 6328 images belonging to 6 classes.


In [8]:
weights = model.get_class_weights(train_generator.classes, model)
model.compile(loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
# model.model.compile(optimizer='adam', loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
# instance_model.compile(optimizer='adam', loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)

In [9]:
model.show_summary()

##### ImageNetOntop_Model #####
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D) 

In [20]:
model.fit_from_generator(path=f'{dataset_path}/{dataset}', 
                         train_generator=train_generator, validation_generator=validation_generator,
                         test_generator=test_generator,
                         evaluate_net=False, use_model_check_point=True, use_early_stop=False, weighted=True,
                         show_activations=False, n_workers=2)

W0111 00:13:13.478097 140217152837440 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


class_weights [2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]
**** Class weights ****
[2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]
*** ** *** *** *** ** ***
Epoch 1/64
894/894 [==============================] - 697s 780ms/step - loss: 1.5142 - categorical_accuracy: 0.3370 - val_loss: 1.3585 - val_categorical_accuracy: 0.4138
Epoch 2/64
894/894 [==============================] - 878s 982ms/step - loss: 1.3335 - categorical_accuracy: 0.4296 - val_loss: 1.2534 - val_categorical_accuracy: 0.4584
Epoch 3/64
894/894 [==============================] - 746s 834ms/step - loss: 1.2823 - categorical_accuracy: 0.4573 - val_loss: 1.2736 - val_categorical_accuracy: 0.4901
Epoch 4/64
894/894 [==============================] - 722s 807ms/step - loss: 1.2452 - categorical_accuracy: 0.4807 - val_loss: 1.2583 - val_categorical_accuracy: 0.4972
Epoch 5/64
894/894 [==============================] - 716s 801ms/step - loss: 1.2098 - categorical_accuracy: 0.4997 - va

OSError: Unable to create file (file write failed: time = Sat Jan 11 04:41:07 2020
, filename = '/home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification//models/ImageNetOntop_Model/2020-01-11__00_13//weights.22-1.07.hdf5', file descriptor = 121, errno = 28, error message = 'No space left on device', buf = 0x1daf47b8, total write size = 96, bytes this sub-write = 96, bytes actually written = 18446744073709551615, offset = 0)

In [12]:
model.model_path

'/home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification//models/PlacesOntop_Model/2020-01-05__18_54/'

In [13]:
model = model.load_model(model.model_path)

In [11]:
model.model_is_trained = True

In [12]:
model.save_model()

weights [2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]
